## 関数テスト用のnotebook

In [1]:
#データの読み込みから実行まで全てを実行するくRunnerクラスを定義する
from typing import Any, Dict, Tuple, Union
import sys
import numpy as np
import joblib
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import Callback,ModelCheckpoint
from tensorflow.keras.layers import Dense,BatchNormalization,Activation
from tensorflow.keras.activations import relu

sys.path.append('../../')
from Utils.utils import *
from Utils.get_q import *

layer_name_list=['batch_normalization1', 'batch_normalization2', 'batch_normalization3','batch_normalization4', 'batch_normalization5',
                  'batch_normalization6', 'batch_normalization7', 'batch_normalization8','batch_normalization9','batch_normalization10',]

CFG = {
  'N': 100,
  'M': 6000,
  'L': 10,
  'ini_type' : 'A',
  'data_name': 'MNIST',


  'data_seed': 4244, #データのサンプリングを固定するためのseed
  'seed1' : 820,
  'seed2' : 314,
  'seed3' : 1228,
}

In [2]:
def preprocessing(X_train, X_test,y_train, y_test) -> np.ndarray:  
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)

        X_train = X_train / 255
        X_test = X_test / 255
        # Use 32-bit instead of 64-bit float
        X_train = X_train.astype("float32")
        X_test = X_test.astype("float32")
        
        idx = np.random.choice(X_train.shape[0], size=6000)
        X_train = X_train[idx]
        y_train = y_train[idx]
        
        X_train_out = X_train
        X_test_out = X_test
        y_train_out = y_train
        y_test_out = y_test
    
        return X_train_out, y_train_out, X_test_out, y_test_out


def PCA_SS(input_train,input_test):
        # Make an instance of the Model
        pca = PCA(n_components=CFG['N'])
        scaler = StandardScaler()

        train_img = pca.fit_transform(input_train)
        train_img =scaler.fit_transform(train_img)
        
        test_img = pca.transform(input_test)
        test_img =scaler.transform(test_img)
        return train_img, test_img, pca, scaler

params= {
  'N': 100,
  'num_classes':10,
  'num_layers':10,
  'activation':'relu',
  'activation_last':'softmax',
  'learning_rate': 0.1,
  'loss': 'sparse_categorical_crossentropy',
  'metric': 'accuracy',
  'random_seed': 777,
  'bias_initializer': tf.keras.initializers.Constant(0.1),
  'optimizer':'adam',
  'epochs':1000,
  'batch_size':256
}

In [3]:
set_seed(CFG['data_seed'])
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()
print('='*15+'Preprocess data'+'='*15)
X_train, y_train, X_test, y_test = preprocessing(X_train, X_test,y_train, y_test)
print('Train size:',X_train.shape, 'Test size:', y_train.shape)
X_train, X_test, pca1, scaler1 = PCA_SS(X_train, X_test)

===============Preprocess data===============
Train size: (6000, 784) Test size: (6000,)


In [4]:
set_seed(CFG['seed1'])
w_intializer1 = tf.keras.initializers.RandomNormal(mean=0, stddev=1)
bias_initializer1 = tf.keras.initializers.Constant(0.1)
model1 = create_fully_model(params=params,W=w_intializer1,B=bias_initializer1)
history1 = model1.fit(X_train, y_train,
                batch_size=params["batch_size"],
                epochs=params["epochs"],
                verbose=1,
                shuffle=True,
                validation_data=(X_test, y_test),
                callbacks=[LogEpochIntermediateCallcack(layer_name_list=layer_name_list,CFG=CFG,X_train=X_train,
                                                        path=f"../Test/Test_output/spinA_{CFG['data_name']}_ini{CFG['ini_type']}_M{CFG['M']}_L{CFG['L']}_seed{CFG['seed1']}.pkl")]
                )

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-12-23 16:07:51.228520: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-23 16:07:51.228782: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


 71/188 [==========>...................] - ETA: 0s

2023-12-23 16:07:51.643556: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-23 16:07:51.693130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


104/188 [===============>..............] - ETA: 0s

...

188/188 [==============================] - 5s 19ms/step


2023-12-23 16:12:50.801530: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 51s 2s/step - loss: 1.5191 - accuracy: 0.5147 - val_loss: 2.0213 - val_accuracy: 0.3406
Epoch 13/1000


2023-12-23 16:12:55.739435: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


23/24 [===========================>..] - ETA: 0s - loss: 1.4664 - accuracy: 0.5355

In [30]:
if CFG['ini_type'] == 'A':
            print('='*15+'ini_type A'+'='*15)
            set_seed(CFG['seed2'])
            w_intializer2 = tf.keras.initializers.RandomNormal(mean=0, stddev=1)
            bias_initializer2 = tf.keras.initializers.Constant(0.1)
elif CFG['ini_type'] == 'B':
    print('='*15+'ini type B'+'='*15)
    w_intializer2 = w_intializer1
    bias_initializer2 = bias_initializer1
    set_seed(CFG['seed3'])

model2 = create_fully_model(params=params,W=w_intializer2,B=bias_initializer2)    
history2 = model2.fit(X_train, y_train,
        batch_size=params["batch_size"],
        epochs=params["epochs"],
        verbose=1,
        shuffle=True,
        validation_data=(X_test, y_test),
        callbacks=[LogEpochIntermediateCallcack(layer_name_list=self.layer_list,CFG=CFG,X_train=X_train,
                                                path=f"../Output/Spin/spinB_{CFG['data_name']}_ini{CFG['ini_type']}_M{CFG['M']}_L{CFG['L']}_seed{CFG['seed2']}.pkl")]
        )

In [32]:
loss1 = joblib.load('../Output/Loss/model001_iniA_M6000_L10_seed820.pkl')